<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import shutil
if os.path.exists('Tutorial_IDP_SAPs'):
    shutil.rmtree('Tutorial_IDP_SAPs')



In [20]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis

fatal: destination path 'Tutorial_IDP_SAPs' already exists and is not an empty directory.


In [ ]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

In [ ]:
ratio='900-0'
ionic_strength='0.3'
peptide_pair='HHHEEEWWW-HWKHWKHWK'

In [ ]:
# First check file
residues_file = 'residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {ratio} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {peptide_pair} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 10000

In [ ]:
#Check in src the file parsers.py, it will show all options available